In [2]:
import pandas as pd
import requests


### Declarando Algumas Funções utilitarias

In [11]:
#Declaração de algumas fuções utilitarias

def download_csv(ulrs_dict, folder = "data"):
        """
        Faz o download de arquivos csv dado um decionário de URL's e salva em uma pasta definida
        Args:
                urls_dict (dic): Dicionário com as URL's de Download. Por padrão a pasta será "data"
                folder (str): Nome da pasta que o arquivo csv deve ser inserido 
        Return:
                void.
        
        """
        try:
                for key in ulrs_dict:
                        req = requests.get(ulrs_dict[key])
                        url_content = req.content
 
                        csv_file = open('{}/{}.csv'.format(folder,key), 'wb')
                        csv_file.write(url_content)
                        csv_file.close()
        except:
                print("Something Wrong!")
                exit()
                
        print("Downloaded!!")

def csv_df(urls_dict, folder="data"):

        """
        Cria dataframes dado uma pasta com os csv's  no seguinte pafrão : [nome_do_csv]_[periodo_de_referencia]
        Args:
                urls_dict (dic): Dicionário com as URL's de Download.
                folder (str): Nome da pasta onde estão os arquivos. Por padrão a pasta será "data"
        Return:
                df (Dataframe): dataframe com a combinação de todos os csv's inseridos
        
        """
        df = pd.DataFrame()

        for key in urls_dict:
                csv_dataframe = pd.read_csv("{}/{}.csv".format(folder, key), sep=";")
                csv_dataframe['semestre'] = key[-6:]
        
                df = df.append(csv_dataframe, ignore_index=True)
                
        print("Converted!")
        return df
    

### Load dos csvs de matrículas

In [12]:
urls_matriculas = {
        "Matriculas_2018_1" : "https://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/3c1feba4-ced1-466e-8e94-a040224a51dc/download/matricula-componente-20181.csv",
        "Matriculas_2018_2" : "https://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/0bfcaf6a-4424-4983-8ba8-d330350a8fbe/download/matricula-componente-20182.csv"
        }

download_csv(urls_matriculas)

matriculas = csv_df(urls_matriculas)



Downloaded!!
Converted!


In [15]:
# Separando Amostra de interesse:

# ID do Curso BCH em Ciências e tecnologia = 10320810
id_cet = 10320810
matriculas_cet = matriculas[matriculas.id_curso == id_cet]


In [16]:
matriculas_cet.head()

,id_turma,discente,id_curso,unidade,nota,reposicao,faltas_unidade,media_final,numero_total_faltas,descricao,semestre
1269,57618131,0a6e5a96bb0c7be8ea7bfd46be7a5184,10320810.0,NaN,NaN,NaN,NaN,NaN,NaN,INDEFERIDO,2018_1
1270,57618131,0a6e5a96bb0c7be8ea7bfd46be7a5184,10320810.0,1.0,"3,0",False,0.0,"6,0",12.0,APROVADO POR NOTA,2018_1
1271,57618131,0a6e5a96bb0c7be8ea7bfd46be7a5184,10320810.0,2.0,"6,5",False,0.0,"6,0",12.0,APROVADO POR NOTA,2018_1
1272,57618131,0a6e5a96bb0c7be8ea7bfd46be7a5184,10320810.0,3.0,"8,5",False,0.0,"6,0",12.0,APROVADO POR NOTA,2018_1
1273,57618131,028467035e9d9396dca378d96f928c46,10320810.0,1.0,"5,0",False,0.0,"6,0",0.0,APROVADO POR NOTA,2018_1


### Load dos csvs das turmas ofertadas

In [18]:
urls_turmas = {
        "Turmas_2018_1" : "https://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/3ae16138-4214-4a30-ac2d-6cffd6237031/download/turmas-2018.1.csv",
        "Turmas_2018_2" : "https://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/77fe7603-0e71-4e21-8cd4-cb823353023f/download/turmas-2018.2.csv"
}

download_csv(urls_turmas)
turmas = csv_df(urls_turmas)

Downloaded!!
Converted!


In [19]:
#Filtrando colunas de interesse 

columns_sel = ['id_turma', 'id_componente_curricular','siape', 'semestre', 'situacao_turma']

turmas = turmas.loc[:, columns_sel]

In [20]:
# Selecionando apenas as turmas de CET
id_turmas_cet = matriculas_cet.id_turma.unique()
turmas_cet = turmas[turmas.id_turma.isin(id_turmas_cet)]

### Load do csv de componentes curriculares 

In [21]:
url_componentes_curriculares = {
        "ComponentesCurriculares" : "https://dados.ufrn.br/dataset/3fea67e8-6916-4ed0-aaa6-9a8ca06a9bdc/resource/9a3521d2-4bc5-4fda-93f0-f701c8a20727/download/componentes-curriculares-presenciais.csv"
        }

download_csv(url_componentes_curriculares)
componentes_curriculares = csv_df(url_componentes_curriculares)

Downloaded!!
Converted!


### Filtragens e manipulações

In [22]:
# Selecionando os componentes curriculares de Cet
cc_ect = componentes_curriculares[componentes_curriculares.unidade_responsavel == 'ESCOLA DE CIÊNCIAS E TECNOLOGIA']

In [23]:
# Juntando com o dataframe de turmas ofertadas 
turmas_cet_cc = turmas_cet.merge(cc_ect, how="inner", left_on="id_componente_curricular", right_on="id_componente")

In [24]:
# Selecionando dados das turmas de interesse
disciplinas_desejadas_nome = ['PRÉ-CÁLCULO', 'VETORES E GEOMETRIA ANALÍTICA', 'CÁLCULO I', 'CÁLCULO II']

turmas_cet_interesse = turmas_cet_cc[turmas_cet_cc.nome.isin(disciplinas_desejadas_nome)]

In [25]:
#Montando dataset com as matrículas das turmas de interesse

matriculas_selecionadas = matriculas_cet.merge(turmas_cet_interesse, on="id_turma")

In [26]:
# Excluindo status indesejáveis
status_indesejados = ['INDEFERIDO', 'EXCLUIDA', 'TRANCADO', 'DESISTENCIA', 'CANCELADO']

matriculas_selecionadas = matriculas_selecionadas[~matriculas_selecionadas.descricao.isin(status_indesejados)]

In [27]:
matriculas_selecionadas.columns

Index(['id_turma', 'discente', 'id_curso', 'unidade', 'nota', 'reposicao',
       'faltas_unidade', 'media_final', 'numero_total_faltas', 'descricao',
       'semestre', 'id_componente_curricular', 'siape', 'semestre_x',
       'situacao_turma', 'id_componente', 'tipo_componente', 'codigo', 'nivel',
       'nome', 'unidade_responsavel', 'ch_teorico', 'ch_pratico', 'ch_estagio',
       'ch_total', 'ch_dedicada_docente', 'ch_ead', 'cr_max_ead',
       'permite_horario_flexivel', 'qtd_unidades', 'procedimentos_avaliacao',
       'equivalencia', 'pre_requisito', 'co_requisito', 'ementa',
       'bibliografia', 'objetivos', 'conteudo', 'competencias_habilidades',
       'referencias', 'ano_programa', 'periodo_programa', 'modalidade',
       'curso_componente', 'semestre_y'],
      dtype='object')

In [28]:
status_aprovado = ['APROVADO', 'APROVADO POR NOTA']

matriculas_selecionadas['reprovou'] = matriculas_selecionadas.descricao.apply(lambda x: False if x in status_aprovado else True)

In [29]:
culunas_de_interesse = ['discente', 'unidade', 'nota', 'reposicao', 'reprovou', 'faltas_unidade', 'media_final', 'numero_total_faltas', 'siape', 'nome']

matriculas_selecionadas = matriculas_selecionadas.loc[:, culunas_de_interesse]
matriculas_selecionadas.rename(columns={'nome' : 'nome_componente'}, inplace=True)

In [35]:
matriculas_selecionadas.head()

,discente,unidade,nota,reposicao,reprovou,faltas_unidade,media_final,numero_total_faltas,siape,nome_componente
1,2b8e11f6c6329ca8022a56803c0e477c,2.0,"0,0",False,True,0.0,"0,2",22.0,1671983.0,CÁLCULO I
2,2b8e11f6c6329ca8022a56803c0e477c,3.0,"0,5",False,True,0.0,"0,2",22.0,1671983.0,CÁLCULO I
3,2b8e11f6c6329ca8022a56803c0e477c,1.0,"0,0",False,True,0.0,"0,2",22.0,1671983.0,CÁLCULO I
5,051ebf8466b8fa12bc1b20c00c6c167b,1.0,"3,3",False,True,0.0,"1,1",18.0,1671983.0,CÁLCULO I
6,051ebf8466b8fa12bc1b20c00c6c167b,2.0,"0,0",False,True,0.0,"1,1",18.0,1671983.0,CÁLCULO I
